<font color="#CA3532"><h1 align="left">**Aprendizaje por Refuerzo**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Multiarmed Bandits Contextuales**</h2></font>

# **Contextual Bandits**

En este notebook se usa la librería SpaceBandits.

In [ ]:
import numpy as np

In [ ]:
# Adaptado de:
# https://github.com/fellowship/space-bandits/releases/tag/v0.0.9

In [ ]:
COLAB = True

In [ ]:
if COLAB:
    from google_drive_downloader import GoogleDriveDownloader as gdd
    gdd.download_file_from_google_drive(file_id='1ZEYZXdC7GyHXqnt5Wl_orkHmc97vWfGO',
                                        dest_path='./spacebandits.zip', unzip=True)

# 1. Ejemplo de prueba

Los modelos basados en contextual bandits son **difíciles de validar** debido a la naturaleza particular de los problemas que tratan de resolver.
La mejor manera de confirmar que un modelo converge en la práctica es construir un ejemplo de prueba donde se pueda evaluar la diferencia entre el reward total conseguido por el modelo y el reward máximo que podría haber conseguido:

$$
{\bf ineficiencia} = {\bf reward\_máximo\_teórico} -
{\bf reward\_conseguido}
$$

Este reward máximo no se puede calcular en la realidad ya que para ello habría que saber con absoluta certeza, para cada caso, cuál sería la ganancia de cada una de las acciones posibles. La acción óptima para cada caso particular sería la que conlleva la máxima ganancia, y el reward máximo sería la suma de las máximas ganancias en cada caso.

Como esto es imposible saberlo en la realidad (si se supiese ya estaría el problema resuelto) lo que se hace es construir un problema de prueba donde por construcción se pueden calcular las ganancias exactas de cada posible acción (y con ello calcular la eficiencia del contextual bandit).

**Es importante remarcar que los algoritmos contextual bandits no necesitan estas ganancias exactas. Donde las necesitamos es en el cálculo de la eficiencia de dichos modelos.**<br> <br>

En este notebook se construye el siguiente problema de prueba:

Tenemos un conjunto de clientes, y de cada cliente tenemos dos datos:

* **Edad**
* **Ingresos generados** hasta el momento por ese cliente (gasto de ese cliente en nuestra empresa).

Nuestra compañía ofrece tres promociones, y el problema es encontrar qué promoción ofrecemos a cada cliente. Una vez que se ofrece una promoción a un cliente, este puede ignorarla o acogerse a ella, generando en este caso una ganancia para la compañía. Por tanto, las **acciones** posibles son las promociones que le podemos ofrecemos al cliente (tres en este ejemplo).

En caso de que un cliente se acoja a una promoción la ganancia para la compañía será:

* **Promoción 0**: ganancia=10
* **Promoción 1**: ganancia=25
* **Promoción 2**: ganancia=100


Hasta aquí son datos que la compañía tiene bajo control.
Sin embargo la compañía desconoce cómo van a responder los clientes a las promociones. Es decir, desconoce la probabilidad de que un cliente $x_i$ se acoja a la promoción $j$ en caso de que se le ofrezca. Es decir, se desconocen las siguientes cantidades:

* $p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,0)$
* $p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,1)$
* $p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,2)$


y la decisión óptima sería, una vez calculadas estas tres cantidades para un cliente, elegir la promoción que conlleve mayor ganancia esperada.

Si supiéramos el valor de estas cantidades para cada cliente (se pueden ver como funciones que dependen del cliente y la promo ofrecida), se podría calcular la ganancia esperada de cada acción (promo ofrecida) como:

* Si se ofrece promo 0:
$$
ganancia\,esperada = 10 \cdot
p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,0)
$$
* Si se ofrece promo 1:
$$
ganancia\,esperada = 25 \cdot
p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,1)
$$
* Si se ofrece promo 2:
$$
ganancia\,esperada = 100 \cdot
p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,2)
$$


Sin embargo, **es imposible** conocer a priori esas cantidades.
La compañía espera que cada cliente responda de una manera diferente a las promociones de acuerdo a sus características, pero la compañía desconoce los segmentos de clientes y cómo responden a las promociones.

**El enorme valor del modelo contextual bandit** está en que dicha técnica va a construir esos segmentos y las funciones que modelan la respuesta de cada cliente a cada promo. Lo va a hacer de manera **online** (incremental, según se van ofreciendo promos a clientes) y de tal forma que va a intentar maximizar la ganancia obtenida en todo el proceso (incluyendo los pasos iniciales de recolección de información).

In [ ]:
import numpy as np
import pandas as pd
from random import random, randint
import matplotlib.pyplot as plt

%config InlineBackend.figure_format='retina'

mean = np.mean

A continuación se definen las funciones que generan los clientes en nuestro ejemplo de prueba. <br> <br>
Por construcción, habrá dos tipos de clientes **(esto no lo sabe el contextual bandit)**:

* **Segmento de clientes A.** Media de edad=25 (std=5), media de ingresos generados=100 (std=15)
* **Segmento de clientes B.** Media de edad=50 (std=6), media de ingresos generados=50 (std=25)

Nuestra compañía ofrece tres promociones: la primera es la mejor para un grupo de clientes, mientras que la tercera es la mejor para el otro grupo. La segunda promoción no es óptima para ninguno de los grupos de clientes. **Todo esto no lo sabe a priori el contextual bandit** (salvo que hay tres promociones). Como en la mayoría de las situaciones de la vida real, un número significativo de clientes no hará compras. Los contextual bandits manejan de manera natural estas situaciones ya que modelan la generación de recompensas como un proceso estocástico.

In [ ]:
def get_cliente(tipo_cliente=None):
    """
       Hay dos variables por cada cliente:

       - Edad
       - Ingresos generados

       Hay a su vez dos tipos de clientes:

        - Tipo A
            - Edad: media=25, std=5, edad mínima=18
            - Ingresos generados: media=100, std=15
        - Tipo B
            - Edad: media=45, std=6
            - Ingresos generados: media=50, std=25
    """
    if tipo_cliente is None:
        if np.random.rand() > .5: # se "lanza una moneda" (coin toss)
            tipo_cliente = "A"
        else:
            tipo_cliente = "B"
    edad = 0
    ingresos_generados = -1
    if tipo_cliente == "A":
        while edad < 18:
            edad = round(np.random.normal(25, 5))
        while ingresos_generados < 0:
            ingresos_generados = round(np.random.normal(100, 15), 2)
    if tipo_cliente == "B":
        while edad < 18:
            edad = round(np.random.normal(45, 6))
        while ingresos_generados < 0:
            ingresos_generados = round(np.random.normal(50, 25), 2)
    return tipo_cliente, np.array([edad, ingresos_generados])


def get_posibles_ganancias(tipo_cliente, edad):
    """
    Hay tres acciones posibles:
        - Promo 0:
            - ganancia=10 si el cliente se acoge
            - ganancia=0 en caso contrario

        - Promo 1:
            - ganancia=25 si el cliente se acoge
            - ganancia=0 en caso contrario

        - Promo 2:
            - ganancia=100 si el cliente se acoge
            - ganancia=0 en caso contrario

    Las probabilidades de aceptación de cada promoción son:
        - Tipo de clientes A:
            - 90% aceptación promo 0 (ganancia promedio=10*90% = 9)
            - 10% aceptación promo 1 (ganancia promedio=25*10% = 2.5)
            -  2% aceptación promo 2 (ganancia promedio=100*2% = 2)

        - Tipo de clientes B:
            - 25% aceptación promo 0 (ganancia promedio=10*25% = 2.5)
            - 10% aceptación promo 1 (ganancia promedio=25*10% = 2.5)
            - 20% aceptación promo 2 (ganancia promedio=100*20% = 20)

        #- Tipo de clientes B:
        #    - 25% aceptación promo 0 (ganancia promedio=10*25% = 2.5)
        #    - 1% aceptación promo 1 (ganancia promedio=25*1% = 0.25)
        #    - 2% aceptación promo 2 (ganancia promedio=100*2% = 2)
    Matriz de ganancias esperadas (ganancias promedio):

           promo0|promo1|promo2
    ----------------------------
    grupoA|  $9  | $2.5 |  $2
    ----------------------------
    grupoB|  $2.5| $2.5 |  $20


    La acción óptima para el grupo A es ofrecerles la promo 0.
    La acción óptima para el grupo B es ofrecerles la promo 2.
    La peor acción es ofrecer la promo 1.

    El contextual bandit desconoce inicialmente que hay diferentes
    tipos de clientes y cuál es su relación con las tasas de
    aceptación, pero esto lo irá descubriendo durante su proceso de
    aprendizaje.
    """
    ganancias_promos = [10,25,100]

    if tipo_cliente == "A":
        promos_que_aceptaria = np.random.rand(3)<[0.9,0.1,0.02]

    #if tipo_cliente == "B":
    else:
        promos_que_aceptaria = np.random.rand(3)<[0.25,0.1,0.2]
        #promos_que_aceptaria = np.random.rand(3)<[0.25,0.01,0.02]

    # posibles_ganancias es un array con 3 componentes.
    # Cada componente es cero si el cliente no aceptaría esa promo,
    # o la ganancia asociada en caso de que la aceptase
    posibles_ganancias = promos_que_aceptaria * ganancias_promos

    return posibles_ganancias

In [ ]:
np.random.rand(3)

In [ ]:
np.random.rand(3)<[0.9,0.1,0.02]

In [ ]:
compra0, compra1, compra2 = (np.random.rand(3)<[0.9,0.1,0.02])

In [ ]:
print(compra0)
print(compra1)
print(compra2)

In [ ]:
n  = 10000
n0 = 0
n1 = 0
n2 = 0

for i in range(n):
    compra0, compra1, compra2 = (np.random.rand(3)<[0.9,0.1,0.02])
    if compra0:
        n0 = n0 + 1
    if compra1:
        n1 = n1 + 1
    if compra2:
        n2 = n2 + 1

In [ ]:
n0/n, n1/n, n2/n

In [ ]:
(np.random.rand()) > 0.5

In [ ]:
np.random.rand(3)<[0.9,0.1,0.02]

In [ ]:
# clase para generar cliente:

class cliente():
    def __init__(self,tipo_cliente=None):

        a, b = get_cliente(tipo_cliente)
        self.tipo_cliente = a
        self.contexto = b

        c = get_posibles_ganancias(self.tipo_cliente, b[0]) # b[0] es edad
        self.posibles_ganancias = c

    def get_reward(self,accion):
        return self.posibles_ganancias[accion]

    def get_context(self):
        return self.contexto

In [ ]:
c = cliente()

In [ ]:
c.get_context() # datos observables

In [ ]:
c.tipo_cliente # esto es interno al cliente, no es observable

In [ ]:
c.posibles_ganancias # esto tampoco es observable

In [ ]:
c.get_reward(0) # reward que obtenemos si lanzamos acción 0 sobre ese cliente

## Visualización de los segmentos de clientes

A continuación visualizaremos los diferentes segmentos de clientes. Se recalca de nuevo que el contextual bandit no dispondrá de esta información sino que la descubrirá en su proceso de aprendizaje.

In [ ]:
groupA = [cliente(tipo_cliente="A") for _ in range(1000)]
groupB = [cliente(tipo_cliente="B") for _ in range(1000)]
plt.scatter([x.get_context()[0] for x in groupA],
            [x.get_context()[1] for x in groupA],
            label='group A', alpha=0.25)
plt.scatter([x.get_context()[0] for x in groupB],
            [x.get_context()[1] for x in groupB],
            label='group B', alpha=0.25)
plt.xlabel('edad', fontsize=14)
plt.ylabel('ingresos generados', fontsize=14)
plt.legend();

## Sanity Check: Ganancias Esperadas

Para asegurarnos de que se están generando bien los datos, a continuación se construye una muestra aleatoria de cada grupo y se calcula la ganancia promedia de cada uno de ellos para cada posible acción (promo). Es otra forma de confirmar qué promos son mejores en cada grupo.

In [ ]:
clientes_tipoA = [cliente(tipo_cliente="A") for _ in range(100000)]
rewards_si_promo0 = [c.posibles_ganancias[0] for c in clientes_tipoA]
rewards_si_promo1 = [c.posibles_ganancias[1] for c in clientes_tipoA]
rewards_si_promo2 = [c.posibles_ganancias[2] for c in clientes_tipoA]
print('ganancias esperadas grupo A (100000 muestras):',
      [mean(rewards_si_promo0), mean(rewards_si_promo1), mean(rewards_si_promo2)])

clientes_tipoB = [cliente(tipo_cliente="B") for _ in range(100000)]
rewards_si_promo0 = [c.posibles_ganancias[0] for c in clientes_tipoB]
rewards_si_promo1 = [c.posibles_ganancias[1] for c in clientes_tipoB]
rewards_si_promo2 = [c.posibles_ganancias[2] for c in clientes_tipoB]
print('ganancias esperadas grupo B (100000 muestras):',
      [mean(rewards_si_promo0), mean(rewards_si_promo1), mean(rewards_si_promo2)])


## El trabajo que tiene que hacer el modelo

El contextual bandit no sabe inicialmente que hay dos tipos de clientes. Su tarea es descubrir estas distribuciones y a la vez caracterizar cómo responden a las diferentes promos. Esta es la naturaleza del problema de los contextual bandits y lo que hace que el problema sea tan difícil. <br> <br>
En una primera aproimación usaremos un modelo de regresión lineal bayesiana que mapea directamente los atributos de cada cliente (en nuestro caso edad e ingresos generados) a una recompensa esperada a través de coeficientes lineales:

$$
{\bf recompensa\_esperada} = \alpha_0 + \alpha_1 \cdot edad + \alpha_2 \cdot ingresos\_generados
$$

Usaremos el aprendizaje "en línea" para este ejemplo, lo que significa que el modelo observa un cliente y decide una acción (qué promo ofrecerle), observa la recompensa y repite el proceso para el siguiente cliente.

In [ ]:
def show_records(records):
    cumm_ideal_rew_plt = np.cumsum(records["ideal_reward_record"])
    cummulative_reward_plt = np.cumsum(records["reward_record"]) # / aux

    ts = records['timesteps']
    plt.plot(ts, cummulative_reward_plt, label='cummulative reward')
    plt.plot(ts, cumm_ideal_rew_plt, label='maximum reward')
    plt.plot(ts, cumm_ideal_rew_plt - cummulative_reward_plt,
             color='red', label='cummulative regret')
    plt.plot([ts[0],ts[-1]], [0,0], ':k')
    plt.title('Cummulative Reward & Regret')
    plt.legend()
    plt.show()
    print("Total reward      :", cummulative_reward_plt[-1])
    print("Total ideal reward:", cumm_ideal_rew_plt[-1])


def show_records_avgs(records):
    # después de hacer el cumsum paso a float porque si no al dividir
    # posteriormente por (i+1) no convierte el array a float sino que
    # se queda en entero
    avg_reward_plt = np.cumsum(records["reward_record"]).astype(float)
    avg_ideal_rew_plt = np.cumsum(records["ideal_reward_record"]).astype(float)

    for i in range(len(avg_reward_plt)):
        avg_reward_plt[i] /= (i+1)
        avg_ideal_rew_plt[i] /= (i+1)

    ts = records['timesteps']
    plt.plot(ts, avg_reward_plt, label='average reward por cliente')
    plt.plot(ts, avg_ideal_rew_plt, label='maximum reward por cliente')
    plt.plot(ts, records['avg_regret_record'],
             color='red', label='mean regret')
    plt.title('Average Reward & Regret')
    plt.legend()
    print("Average reward      :", avg_reward_plt[-1])
    print("Average ideal reward:", avg_ideal_rew_plt[-1])


### Paso a paso:

In [ ]:
# conda install tensorflow==1.15

In [ ]:
from space_bandits import LinearBandits

n_acciones = 3 # número de promociones diferentes entre las que debemos
               # escoger una para el cliente
n_features = 2 # edad e ingresos obtenidos con ese cliente

modelo_lineal = LinearBandits(n_acciones, n_features,
                              initial_pulls=100)

In [ ]:
modelo_lineal.mu

In [ ]:
c = cliente()
c.get_context()

In [ ]:
c.posibles_ganancias

In [ ]:
modelo_lineal.expected_values(c.get_context())

In [ ]:
accion = modelo_lineal.action(c.get_context())
#accion = 0
accion

In [ ]:
modelo_lineal.mu

In [ ]:
10.12632168 + 28*(-0.18626279) + 122.98*0.03114483
-5.88232122 + 28*0.09572589 + 122.98*0.06857028

In [ ]:
accion

In [ ]:
reward = c.get_reward(accion)
reward

In [ ]:
modelo_lineal.update(c.get_context(), accion, reward)

In [ ]:
modelo_lineal.mu

In [ ]:
def iterate_model(modelo, acciones_ideales, steps, records=None,
                  plot_frequency=250, avg_length=150,
                  random_state=None):
    """Goes through online learning simulation with model."""

    if random_state is not None:
        np.random.seed(random_state)

    #these will track values for plotting
    if records is None:
        records = dict()
        records['timesteps'] = []
        records['reward_record'] = []
        records['ideal_reward_record'] = []
        records['regret_record'] = []
        records['avg_regret_record'] = []
        start = 0
    else:
        start = records['timesteps'][-1] + 1
    for i in range(start, start+steps):
        records['timesteps'].append(i)

        c = cliente() # <- cliente que llega

        contexto = c.get_context() # datos del cliente a los que se tiene acceso

        accion = modelo.action(contexto) # acción recomendada por el modelo

        # se lanza la acción sobre el cliente y eso produce una ganancia (reward)
        reward = c.get_reward(accion) # <- ganancia (reward) obtenida tras lanzar la acción

        modelo.update(contexto, accion, reward) # actualización del modelo
        records['reward_record'].append(reward)


        # acción ideal: es aquella que se debería seleccionar
        # si conociéramos el tipo de cliente y el comportamiento
        # de ese cliente para cada promo.
        # ** El agente desconoce todas esas cosas **
        accion_ideal        = acciones_ideales[c.tipo_cliente]
        reward_accion_ideal = c.posibles_ganancias[accion_ideal]
#        reward_ideal_choice = max(c.posibles_ganancias)
        records['ideal_reward_record'].append(reward_accion_ideal)

        #regret is the opportunity cost of not choosing the optimal promotion
        regret = reward_accion_ideal - reward
        records['regret_record'].append(regret)


        #plot occasionally
        if i <= avg_length:
            if i < avg_length:
                moving_avg=0
            else:
                moving_avg = np.array(records['regret_record']).mean()
            if i == avg_length:
                records['avg_regret_record'] = [moving_avg]*avg_length
        else:
            moving_avg = sum(records['regret_record'][-avg_length:])/avg_length
        records['avg_regret_record'].append(moving_avg)
        if (i+1) % plot_frequency == 0 and i > 0:
            show_records(records)
            plt.show()
    return records

## Modelos lineales

In [ ]:
from space_bandits import LinearBandits

n_acciones = 3 # número de promociones diferentes entre las que debemos
               # escoger una para el cliente
n_features = 2 # edad e ingresos obtenidos con ese cliente

modelo_lineal = LinearBandits(n_acciones, n_features,
                              initial_pulls=100)

acciones_ideales = {"A":0, # para el tipo de cliente A lo óptimo es
                           # ofrecerle la promo 0 (mayores ganancias
                           # promedio)

                    "B":2  # para el tipo de cliente B lo óptimo es
                           # ofrecerle la promo 2 (mayores ganancias
                           # promedio)
                   }

records = iterate_model(modelo_lineal, acciones_ideales,
                        steps=400, plot_frequency=400,
#                        random_state=1
                       )


In [ ]:
show_records_avgs(records)

**Optimización realizada por el contextual bandit**

Observamos un alto regret (coste de oportunidad perdida = reward máximo menos reward obtenido) al comienzo del proceso, pero después de unas iteraciones baja significativamente.

## Grabar/Cargar el modelo

Podemos guardar y cargar modelos libremente y continuar aprendiendo después de volver a cargar (como se hace a continuación). Esta es una funcionalidad importante para un modelo en línea.

In [ ]:
#test linear model saving/loading
from space_bandits import load_linear_model
modelo_lineal.save('test_path.pkl')
modelo_lineal = load_linear_model('test_path.pkl')

In [ ]:
#continue training

records = iterate_model(modelo_lineal, acciones_ideales,
                        steps=400, plot_frequency=400,
                        records=records)

In [ ]:
show_records_avgs(records)

## Visualización de la Frontera de Decisión

Después de converger el modelo recomendará una promo diferente a cada cliente dependiendo de dónde esté en el espacio de atributos (edad, ingresos generados). Podemos visualizar esto como una "frontera de decisión".

In [ ]:
def plot_decision_boundary(model, X, Y, h=1, scale=1., parallelize=True,
                           title='decision boundary', thompson=False,
                           classic=False, n_threads=-1, flip_colors=False,
                           alpha=0.5):
    ftnames = X.columns[0], X.columns[1]
    X = X.values
    #model.fit(X[:, :2], Y)
    x_min = X[:, 0].min() - .5 # mínimo de primera variable (edad) - 0.5
    x_max = X[:, 0].max() + .5 # máximo de primera variable (edad) + 0.5
    y_min = X[:, 1].min() - .5 # mínimo de segunda variable (IG) - 0.5
    y_max = X[:, 1].max() + .5 # máximo de segunda variable (IG) + 0.5
    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, h), # *A -> array que va de x_min a x_max de h en h
        np.arange(y_min, y_max, h)  # *B -> array que va de y_min a y_max de h en h
    )
    # meshgrid calcula todas las combinaciones posibles (xx,yy) donde el valor de xx
    # es uno de los valores de *A, y el valor de yy es uno de los valores de *B
    # Es decir, (xx, yy) son todos los centros de los posibles cuadraditos que
    # vamos a dibujar

    if classic:
        Z = model.classic_predict(np.c_[xx.ravel(), yy.ravel()]/scale, thompson=thompson)
    else:
        # predecir la acción a realizar en cada posible combinación (xx,yy), es decir,
        # en cada uno de los posibles cuadraditos:
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()]/scale,
                          thompson=thompson,
                          parallelize=parallelize)

    # Put the result into a color plot.
    Z = Z.reshape(xx.shape)
    plt.pcolormesh(xx, yy, Z, alpha=.25) # dibujo, para cada cuadradito, un color diferente
    # en función de la acción a realizar ahí (Z)

    # Add the training points to the plot.
    if flip_colors:
        Y = np.where(np.array(Y)==1, 0, 1)
    plt.scatter(X[:, 0], X[:, 1], c=Y, alpha=alpha);
    ###plt.scatter(X[:, 0], X[:, 1], c='black', alpha=.1);


    plt.xlabel(ftnames[0], fontsize=14)
    plt.ylabel(ftnames[1], fontsize=14)

    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max());
    plt.title(title)
    plt.show()

In [ ]:
X = groupA + groupB
Y = len(groupA)*[1] + len(groupB)*[0]
edades             = [x.get_context()[0] for x in X]
ingresos_generados = [x.get_context()[1] for x in X]
df = pd.DataFrame({"edad":edades,
                   "ingresos_generados":ingresos_generados})
X = df
X.sample(5)

In [ ]:
%%time
plot_decision_boundary(modelo_lineal, X, Y, h=.5,
                       thompson=False,
                       title='Predicciones del modelo')

**Nota:** el color cian corresponde a promo1, que no es óptima en ningún caso.

### Interpretación del modelo lineal construido

El multiarmed bandit contextual lineal está construyendo las siguientes estimaciones lineales (una para cada acción posible):

$$
\hat Q_{a=0}(contexto) = \alpha_{0,a=0} + \alpha_{1,a=0} \cdot Edad + \alpha_{2,a=0} \cdot IG
$$
$$
\hat Q_{a=1}(contexto) = \alpha_{0,a=1} + \alpha_{1,a=1} \cdot Edad + \alpha_{2,a=1} \cdot IG
$$
$$
\hat Q_{a=2}(contexto) = \alpha_{0,a=2} + \alpha_{1,a=2} \cdot Edad + \alpha_{2,a=2} \cdot IG
$$

La interpretación por ejemplo de $Q_{a=0}(contexto)$ es la ganancia esperada de la acción 0 sobre un cliente del cual sabemos el contexto (en nuestro caso, la edad y el IG).

Si $n_{acciones}$ es el número de posibles acciones, y $n_{features}$ el número de variables de contexto, el modelo construirá $n_{acciones}$ modelos lineales y cada uno de ellos tendrá un término constante y $n_{features}$ coeficientes.
En total, habrá $n_{acciones} · (1+n_{features})$ coeficientes en todo el modelo.

In [ ]:
auxs = []
for a in range(n_acciones):
    aux = "Q_estimada(acción={}) = {:.2f}{}{:.2f}·Edad{}{:.2f}·IG"
    pars = modelo_lineal.mu[a]
    mas0 = ' + ' if pars[0]>=0 else ' '
    mas1 = ' + ' if pars[1]>=0 else ' '
    aux2 = aux.format(a,
                      modelo_lineal.mu[a][2], mas0,
                      modelo_lineal.mu[a][0], mas1,
                      modelo_lineal.mu[a][1])
    print(aux2)
    aux2 = aux2.replace("Edad","{:.2f}")
    aux2 = aux2.replace("IG","{:.2f}")
    auxs.append(aux2)

In [ ]:
c = cliente()
contexto = c.get_context()

aux = "Contexto cliente (Edad,IG) = ({:.2f},{:.2f})\n"
print(aux.format(contexto[0], contexto[1]))
print("Qs estimadas por el modelo (método expected values):")
Qs = modelo_lineal.expected_values(contexto)
for a in range(n_acciones):
    print("  * Q_estimada(acción={}) = {}".format(a,Qs[a].round(3)))
print()
Edad,IG = contexto
print("Qs estimadas a mano con los coeficientes del modelo:")
alphas = modelo_lineal.mu
for a in range(n_acciones):
    Q = alphas[a][2] + alphas[a][0]*Edad + alphas[a][1]*IG
    print("  * "+auxs[a].format(Edad,IG)+" = "+str(Q.round(3)))

print()
print("Acción elegida por el modelo:", modelo_lineal.predict(contexto, thompson=False))